In [1]:
import requests
import csv 
from collections import Counter
from pandas import *

In [2]:
listoffiles=[]
listoffilmnames=[]

def csvgetter(user,password,*otherusernames):
    #first create a session 's'
    s=requests.session()
    #this is the data that is sent to login to icheckmovies
    login_data={'login[username]': user,'login[password]': password,'submit': 'login',}
    #here we send the login data, using the session created before
    s.post("https://www.icheckmovies.com/login/", data=login_data) 
    #once again, whithin the session we get the csv link and call it 'r'
    usernames_list=list(otherusernames)
    usernames_list.append(user)
    for item in usernames_list:
        url='https://www.icheckmovies.com/movies/checked/?user='+item+'&export'
        r=s.get(url)
        checklist=r.text
        checklist_encode = checklist.encode('utf-8')
        filename=item+'_checklist'+'.csv'
        file=open(filename, "w")
        file.write(checklist_encode)
        file.close() 
        listoffiles.append(filename)
    numberofusers=str((len(listoffiles)))
    print('There are '+numberofusers+' users in the database: '+'\n'+'\n'.join(listoffiles))
    
def listmaker(username):
    filmlist=[]
    toplist=[]
    mostchecked=[]
    csvname=username+"_checklist.csv"
    csv_reader = csv.reader(open(csvname, "rb"), delimiter=';')
    for row in csv_reader:
        filmlist.append(row[0])
        mostchecked.append(row[3])
        toplist.append(row[5])        
    filmlist.pop(0)
    mostchecked.pop(0)
    toplist.pop(0)
    compiled_dataframe={'title' : filmlist,
            'most_checked': mostchecked,
            'top_lists': toplist}
    userlist=DataFrame(compiled_dataframe)
    return(userlist)

def multiname(*usernames):
    for name in usernames:
        listmaker(name)
        listoffilmnames.append(filmlist)
        print(name)

In [7]:
username1='Haessar'
username2='hermesmovies'
username3='James88'
password='widmerpool88'

In [8]:
csvgetter(username1,password,username2,username3)

There are 6 users in the database: 
Haessar_checklist.csv
James88_checklist.csv
hermesmovies_checklist.csv
hermesmovies_checklist.csv
James88_checklist.csv
Haessar_checklist.csv


In [9]:
user1_list=listmaker(username1)
user2_list=listmaker(username2)
user3_list=listmaker(username3)
print("user1 is "+username1+", user2 is "+username2+" and user3 is "+username3+", user#_list is a dataframe of each users list")

user1 is Haessar, user2 is hermesmovies and user3 is James88, user#_list is a dataframe of each users list


Want to make a list that both james and will have seen that I haven't seen

In [17]:
#join will and james
combo_2_3=user2_list.append(user3_list)
#return duplicates
seenby_2_3=combo_2_3[combo_2_3.duplicated('title')==True] 
combo_1_2_3=combo_2_3.append(user1_list)  #Create combo of all 3
seenby_1_2_3=combo_1_2_3[combo_1_2_3.duplicated('title')==True] # Show films that all 3 have seen.

In [61]:
#Attempt to find only those films not seen by all.
rec=seenby_2_3.append(seenby_1_2_3) 
nrec=rec.drop_duplicates('title')
nrec['most_checked'] = [ int(x) for x in nrec['most_checked'] ] #NEED TO CONVERT most_checked AND top_lists COLUMNS TO NUMERIC
nrec['most_checked'].convert_objects(convert_numeric=True)



F:\Users\William\Anaconda\lib\site-packages\IPython\kernel\__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [50]:
nrec.sort('most_checked',ascending=False) #CURRENTLY CAN'T SORT PROPERLY DUE TO NON-NUMERIC.

,most_checked,title,top_lists
527,999,East Is East,2
442,9946,The Waterboy,0
599,98828,Pulp Fiction,31
587,98658,Back to the Future,21
242,9858,Contagion,1
22,9758,Mean Streets,17
526,97219,Pirates of the Caribbean: The Curse of the Bla...,10
10,96167,"Monsters, Inc.",12
362,9587,Thunderball,3
156,9534,Road Trip,1


In [53]:
#this is working on a function that will do everything
def superfunction(usr1,pssword,*usrs):
    username1=usr1
    username2=usrs[0]
    username3=usrs[1]
    password=pssword
    csvgetter(username1,password,username2,username3)
    multiname(username1,username2,username3)